In [ ]:
import spacy
import json
from spacy.tokens import DocBin

## Datasets

In [ ]:
path_data_training= '/resources/datasets/unified/training_.json'
path_data_validation = '/resources/datasets/unified/validation_.json'
path_data_testing = '/resources/datasets/unified/test.json'

In [ ]:
with open(path_data_training) as f:
    training_data = json.load(f)

with open(path_data_validation) as f:
    validation_data = json.load(f)

with open(path_data_testing) as f:
    testing_data = json.load(f)

In [ ]:
print(f'Train: {len(training_data)}\nValidation: {len(validation_data)}\nTest: {len(testing_data)}')

#### Crear DocBin

In [ ]:
def make_docbin(data, outputFile):
    """
    data: .json
    outputFile: str
    """
    nlp = spacy.blank("es") 
    db = DocBin()
    for text, annot in data:
        doc = nlp.make_doc(text) 
        ents = []
        for element in annot["entities"]:
            for start, end, label in [element]:
                span = doc.char_span(start,end,label=label,alignment_mode="contract")
                if span is None:
                    print("Skipping entity")
                else:
                    ents.append(span)
        doc.ents = ents
        db.add(doc)
    db.to_disk(f"{outputFile}.spacy")
    return f"Processed {len(db)}"

In [ ]:
output_train = '/resources/datasets/docbin/train'
output_val = '/resources/datasets/docbin/val'
output_test = '/resources/datasets/docbin/test'

In [ ]:
# Create docBin Train
make_docbin(training_data, output_train)

In [ ]:
# Create docBin Val
make_docbin(validation_data, output_val)

In [ ]:
# Create docBin test
make_docbin(testing_data, output_test)

## Train CPU

#### Download base model

In [ ]:
!python -m spacy download es_core_news_lg 

#### Create config

In [ ]:
! python -m spacy init config config.cfg --lang es --pipeline ner --optimize accuracy

#### Training model

In [ ]:
! python -m spacy train config.cfg --output /src/ia2/ia2/models  --paths.train /resources/datasets/docbin/train.spacy --paths.dev /resources/datasets/docbin/val.spacy

## Train GPU

#### Create config

In [ ]:
# ! python -m spacy init config config_GPU.cfg --lang es --pipeline ner --gpu

#### Training model

In [ ]:
# ! python -m spacy train config_GPU.cfg --output /src/ia2/ia2/models  --paths.train /resources/datasets/docbin/train.spacy  --paths.dev /resources/datasets/docbin/val.spacy --gpu-id 0

## Load Best Model

In [ ]:
nlp_ner = spacy.load('/src/ia2/ia2/models/model-best')

In [ ]:
nlp_ner.get_pipe('ner').labels

In [ ]:
nlp_ner.pipeline

## Evaluate

### Evaluate Cli

In [ ]:
! python -m spacy evaluate /src/ia2/ia2/models/model-best  /resources/datasets/docbin/test.spacy --output /resources/ouputs/metrics/metics.json --gold-preproc --displacy-path /resources/ouputs/displacy --displacy-limit 10

#### Evaluate old

In [ ]:
from spacy.scorer import Scorer
from spacy.training import Example
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

def evaluate(model, data):
    examples = []
    scorer = Scorer()
    for text, annotations in data:
        doc = model.make_doc(text)
        example = Example.from_dict(doc, annotations)
        example.predicted = model(str(example.predicted))
        examples.append(example)
    return scorer.score(examples)

def metrics_per_ents(model,data):
    metrics = evaluate(model, data)
    return pd.DataFrame.from_dict(metrics['ents_per_type'])

def metrics(model,data):
    metrics = evaluate(model, data)
    return {'precision': metrics['ents_p'], 'recall': metrics['ents_r'],'f-score': metrics['ents_f']}

In [ ]:
metrics(nlp_ner,testing_data)

In [ ]:
metrics_per_ents(nlp_ner,testing_data)

#### Display Text

In [ ]:
text = testing_data[10][0]
doc = nlp_ner(text)

In [ ]:
doc.ents

In [ ]:
spacy.displacy.render(doc, style='ent', jupyter=True)